In [7]:
# Initial imports
import os
import requests
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
import warnings
import random 
warnings.filterwarnings('ignore')

%matplotlib inline

In [8]:
load_dotenv()

True

In [11]:
import os
import pandas as pd
from alpaca_trade_api.rest import REST, TimeFrame
from dotenv import load_dotenv
import warnings
warnings.filterwarnings('ignore')


def fetch_stock_data(api_key, secret_key, start_date, end_date, tickers, timeframe="1Day"):
    # Set Alpaca API credentials
    alpaca_api_key = os.getenv(api_key)
    alpaca_secret_key = os.getenv(secret_key)

    # Initialize Alpaca API
    api = REST(alpaca_api_key, alpaca_secret_key, base_url="https://paper-api.alpaca.markets")

    # Convert start_date and end_date to ISO format
    start_date_iso = pd.Timestamp(start_date, tz="America/New_York").isoformat()
    end_date_iso = pd.Timestamp(end_date, tz="America/New_York").isoformat()

    # Get stock data using Alpaca API
    df_ticker = api.get_bars(tickers, TimeFrame.Day, start=start_date_iso, end=end_date_iso).df

    # Extract and process data for each ticker
    dfs = {}
    for ticker in tickers:
        ticker_df = df_ticker.loc[df_ticker.symbol == ticker].drop('symbol', axis=1)
        ticker_df = ticker_df['close']
        dfs[ticker] = ticker_df

    # Concatenate dataframes
    new_df = pd.concat(dfs.values(), axis=1, keys=tickers)

    return new_df

# Where you can input your preferred timeframe and stocks
api_key = "ALPACA_API_KEY"
secret_key = "ALPACA_SECRET_KEY"
start_date = "2017-05-01"
end_date = "2020-05-01"
tickers = ["MSFT", "KO","ENB","GOOG"] #can add as many stocks as you want

#result_df = fetch_stock_data(api_key, secret_key, start_date, end_date, tickers)
#this new dataframe will hold a summary of your stocks closing price
result_df = fetch_stock_data(api_key, secret_key, start_date, end_date, tickers)
result_df.head()

,MSFT,KO,ENB,GOOG
timestamp,,,,
2017-05-01 04:00:00+00:00,69.41,43.22,41.09,912.57
2017-05-02 04:00:00+00:00,69.30,43.39,41.51,916.44
2017-05-03 04:00:00+00:00,69.08,43.32,41.44,927.04
2017-05-04 04:00:00+00:00,68.81,43.66,40.50,931.66
2017-05-05 04:00:00+00:00,69.00,43.69,41.43,927.13


## Function 3 - create a Function that Randomly generate weights to each assets in portfolio

- Input = number of assets, number of simulations
- Output = 3 assets, 4 simulation = [[0.2, 0.2, 0.6], [0.3, 0.3, 0.4], [0.5, 0.1, 0.4], [0.8, 0.1, 0.1]]

In [23]:
def weights(stock_list, num_simulations):
    all_lists = []
    num_assets = len(stock_list)
    for _ in range(num_simulations):
        random_weights = gen_random_weights(num_assets)
        all_lists.append(random_weights)
    if num_simulations == 1:
        all_lists = all_lists[0]
    return all_lists
def gen_random_weights(num_weights):
    weights = [random.random() for _ in range(num_weights)]
    normalized_weights = [round(w/sum(weights), 4) for w in weights]
    if sum(normalized_weights) != 1:
        return gen_random_weights(num_weights)
    return normalized_weights



### Function 4: Create Return vs. risk graph to show efficient frontier
 - Input = generated weights list, df
 - Output = Dataframe of portfolio returns and risks per each randomly generated weights

- Eventually, graph using portfolio returns (y axis) vs. portfolio risks (x axis)

W